<a href="https://colab.research.google.com/github/kamusone/BITCOIN-PR-DICTION-UTILIZATION-SENTIMENT-ANALYSE/blob/master/Test_Analyse_sentiment_VM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [18]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install numpy
!pip install -q pandas==0.24.1
!pip install -q seaborn==0.9
!pip install -q pyspark==2.3.3
!pip install -q findspark
!pip install -q handyspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import os
import numpy as np
import findspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
# from handyspark import *

findspark.init()
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] pyspark-shell'


spark = SparkSession.builder.getOrCreate()

sql = SQLContext(spark)

In [21]:
!pip install tweet-preprocessor
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(text):
    text=str(text)
    analyzer = SentimentIntensityAnalyzer()
    text_vs = analyzer.polarity_scores(text)
    return ([text_vs['neg'], text_vs['neu'], text_vs['pos'], text_vs['compound']])

In [22]:
df1 = pd.read_csv('/content/drive/My Drive/sentiment_analysis/tweets_drop.csv', sep = ',')
df1.head()

,timestamp,text
0,2016-06-25 23:59:52,Bitcoin: Pissed-off customers sue GAW Miners i...
1,2016-06-25 23:59:36,I'm not gone lie I be staring at pplwhat btc's...
2,2016-06-25 23:59:17,Did you miss Beyond Bitcoin Hangout? I CANNOT ...
3,2016-06-25 23:59:15,What are the merits and risks of investing in ...
4,2016-06-25 23:59:08,Hash Ocean #Bitcoin Cloud Mining №1 in the wor...


In [47]:
df2 = df1[-100000:]
df2.head()

,timestamp,text
11399482,2019-01-19 14:55:22,.\n\nクス
11399483,2019-01-19 14:55:21,1/2\nTop 5 Cryptocurrency\n # Name Volu...
11399484,2019-01-19 14:55:20,Valor total do Bitcoin enviado para mercados d...
11399485,2019-01-19 14:55:18,Crypto will never be accepted until the indust...
11399486,2019-01-19 14:55:14,I will support him and try to expand the news ...


In [0]:
tweets=sql.createDataFrame(df2)

In [26]:
tweets.show(3)

+-------------------+--------------------+
|          timestamp|                text|
+-------------------+--------------------+
|2019-01-16 11:32:39|I've just posted ...|
|2019-01-16 11:32:33|RT andi_staub: Th...|
|2019-01-16 11:32:31|Huge day today fo...|
+-------------------+--------------------+
only showing top 3 rows



In [49]:
! pip install langdetect
from langdetect import detect

def langdetection(cleaned):
    try :
        if detect(cleaned) == 'en':
            return cleaned
        else :
            return np.nan
    except:
        return np.nan

In [0]:
import preprocessor as p
import re
def clean_tweets(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())

In [51]:
func_udf = udf(clean_tweets, StringType())
tweets = tweets.withColumn('cleaned', func_udf(tweets['text']))
tweets.show(3)

+-------------------+--------------------+--------------------+
|          timestamp|                text|             cleaned|
+-------------------+--------------------+--------------------+
|2019-01-19 14:55:22|               .

クス|                    |
|2019-01-19 14:55:21|1/2
Top 5 Cryptoc...|1 2 Top 5 Cryptoc...|
|2019-01-19 14:55:20|Valor total do Bi...|Valor total do Bi...|
+-------------------+--------------------+--------------------+
only showing top 3 rows



In [52]:
func_udf = udf(langdetection, StringType())
tweets= tweets.withColumn('lang_detect', func_udf(tweets['cleaned']))

tweets.show(3)

+-------------------+--------------------+--------------------+--------------------+
|          timestamp|                text|             cleaned|         lang_detect|
+-------------------+--------------------+--------------------+--------------------+
|2019-01-19 14:55:22|               .

クス|                    |                 NaN|
|2019-01-19 14:55:21|1/2
Top 5 Cryptoc...|1 2 Top 5 Cryptoc...|1 2 Top 5 Cryptoc...|
|2019-01-19 14:55:20|Valor total do Bi...|Valor total do Bi...|                 NaN|
+-------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [0]:
func_udf = udf(sentiment_scores, ArrayType(FloatType()))
tweets = tweets.withColumn('p_neg', func_udf(tweets['cleaned'])[0])
tweets = tweets.withColumn('p_neu', func_udf(tweets['cleaned'])[1])
tweets = tweets.withColumn('p_pos', func_udf(tweets['cleaned'])[2])
tweets = tweets.withColumn('p_comp', func_udf(tweets['cleaned'])[3])

In [0]:
tweets_final = tweets.dropna(subset=['lang_detect'])

In [0]:
tweets_final = tweets_final.selectExpr("timestamp","p_neg","p_neu","p_pos","p_comp")

In [0]:
#tweets_final.repartition(1).write.format('com.databricks.spark.csv').save("C:/Users/kamus/tweets_final1.csv",header = 'true')

In [56]:
tweets_final.show(3)

+-------------------+-----+-----+-----+------+
|          timestamp|p_neg|p_neu|p_pos|p_comp|
+-------------------+-----+-----+-----+------+
|2019-01-19 14:55:22|  0.0|  0.0|  0.0|   0.0|
|2019-01-19 14:55:21|  0.0|0.921|0.079|0.2023|
|2019-01-19 14:55:20|  0.0|  1.0|  0.0|   0.0|
+-------------------+-----+-----+-----+------+
only showing top 3 rows



In [0]:
import json
json_tweets = tweets_final.toJSON().map(lambda j: json.loads(j)).collect()

In [36]:
json_tweets

PythonRDD[39] at RDD at PythonRDD.scala:52

In [0]:
df10k = tweets_final.toPandas()

In [46]:
for i in json_tweets:
  print (i["timestamp"])

2019-01-16 11:32:39
2019-01-16 11:32:33
2019-01-16 11:32:31
2019-01-16 11:32:28
2019-01-16 11:32:27
2019-01-16 11:32:27
2019-01-16 11:32:26
2019-01-16 11:32:22
2019-01-16 11:32:22
2019-01-16 11:32:21
2019-01-16 11:32:19
2019-01-16 11:32:16
2019-01-16 11:32:14
2019-01-16 11:32:14
2019-01-16 11:32:13
2019-01-16 11:32:12
2019-01-16 11:32:08
2019-01-16 11:32:08
2019-01-16 11:32:07
2019-01-16 11:32:06
2019-01-16 11:32:05
2019-01-16 11:32:05
2019-01-16 11:32:00
2019-01-16 11:31:58
2019-01-16 11:31:55
2019-01-16 11:31:55
2019-01-16 11:31:55
2019-01-16 11:31:54
2019-01-16 11:31:54
2019-01-16 11:31:51
2019-01-16 11:31:50
2019-01-16 11:31:49
2019-01-16 11:31:49
2019-01-16 11:31:44
2019-01-16 11:31:43
2019-01-16 11:31:41
2019-01-16 11:31:40
2019-01-16 11:31:37
2019-01-16 11:31:37
2019-01-16 11:31:36
2019-01-16 11:31:33
2019-01-16 11:31:33
2019-01-16 11:31:33
2019-01-16 11:31:33
2019-01-16 11:31:30
2019-01-16 11:31:30
2019-01-16 11:31:30
2019-01-16 11:31:30
2019-01-16 11:31:29
2019-01-16 11:31:28


In [0]:
df10k.head()